In [10]:
import pandas as pd
import myconstants as c

NUM_GAMES = c.NUM_GAMES_2020_21
MIN_GAMES = NUM_GAMES * 0.70

p_df = pd.read_csv('../data/years/2020-21/clean/2020-21_Player_PerGame.csv')
t_df = pd.read_csv('../data/years/2020-21/clean/2020-21_Team_Standings.csv')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# players who haven't played enough games don't qualify
p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 2*p_df['PTS'] + 2*p_df['TRB'] + 2*p_df['AST'] + 4*p_df['STL'] + 4*p_df['BLK']
p_df = p_df.sort_values('PlayerScore', ascending=False)

p_df.head(50)
shortened_t_df = t_df[['TeamCode', 'OverallW', 'OverallL']].rename(columns={'TeamCode': 'Team', 'OverallW': 'Wins', 'OverallL': 'Losses'})
p_df
#pd.merge(p_df, shortened_t_df, on='Team')

,Player,PlayerID,Tms,Pos,Age,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PlayerScore
184,Giannis Antetokounmpo,antetgi01,MIL,PF,26,61,61,33.0,10.3,18.0,...,1.6,9.4,11.0,5.9,1.2,1.2,3.4,2.8,28.1,99.6
399,Nikola Jokić,jokicni01,DEN,C,25,72,72,34.6,10.2,18.0,...,2.8,8.0,10.8,8.3,1.3,0.7,3.1,2.7,26.4,99.0
452,Russell Westbrook,westbru01,WAS,PG,32,65,65,36.4,8.4,19.0,...,1.7,9.9,11.5,11.7,1.4,0.4,4.8,2.9,22.2,98.0
339,Luka Dončić,doncilu01,DAL,PG,21,66,66,34.3,9.8,20.5,...,0.8,7.2,8.0,8.6,1.0,0.5,4.3,2.3,27.7,94.6
231,James Harden,hardeja01,"HOU,BRK",PG,31,44,43,36.6,7.8,16.7,...,0.8,7.1,7.9,10.8,1.2,0.8,4.0,2.3,24.6,94.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Noah Vonleh,vonleno01,BRK,PF,25,4,0,2.8,0.0,0.8,...,0.0,0.3,0.3,0.3,0.0,0.0,0.5,0.5,0.0,1.2
488,Theo Pinson,pinsoth01,NYK,SG,25,17,0,2.0,0.1,0.5,...,0.0,0.3,0.3,0.1,0.0,0.0,0.1,0.2,0.1,1.0
30,Ashton Hagans,haganas01,MIN,PG,21,2,0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
531,Will Magnay,magnawi01,NOP,C,22,1,0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
